In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
ct = np.arange(7*5*2).reshape(7,5,2) #(7, 5, 2)
np.transpose(ct).shape #(2, 5, 7)
# m*n -> transpose -> n*m
# i*j*k -> transpose -> ikj, jik, jki, kij, kji... (6가지)
np.transpose(ct, [1,2,0]).shape #(5, 2, 7)

In [ ]:
sess = tf.Session()

In [ ]:
t = [i for i in range(1,25)]
t = tf.reshape(t,[2,3,4])
t = t[-1]
print(sess.run(t))

In [ ]:
#case study 1 : 네 글자로 구성된 단어 -> 학습
#단어의 앞 3글자 입력 -> 마지막 글자 예측
# wood -> 학습 -> 모델
# woo 입력 -----> 모델 -> d 예측
# wop 입력 -----> 모델 -> d 예측 (오타가 있을때도 예측 가능)

char_arr = [chr(i) for i in range(97,123)]  #'a':97 ~ 'z':122, chr(97) -> 'a'
char_arr  
num_dic = {n:i for i,n in enumerate(char_arr)}  #{"a":0, 'b':1...'z':25}
dic_len = len(num_dic)

In [ ]:
seq_data = ['word','wood','deep','dive','cold','cool','load','love','kiss','kind']
# x:wor -> y:d
# x:woo -> y:d

In [ ]:
def make_batch(seq_data):
    input_batch=[]
    target_batch=[]
    for seq in seq_data: # seq = 'word'
        myInput = [num_dic[n] for n in seq[:-1]] # wor => 22,14,17
#         print(myInput)
        target = num_dic[seq[-1]]
        #원핫 인코딩
        input_batch.append(np.eye(dic_len)[myInput]) #대각 행렬 만드는 함수
        target_batch.append(target)
        
    return input_batch, target_batch

In [ ]:
input_batch, target_batch = make_batch(seq_data)
#  woo           d

In [ ]:
print(np.eye(dic_len)) #대각 행렬 만드는 함수

In [ ]:
np.shape(input_batch) #(10,3,26)
#10: 단어 개수, 3:3글자, 26:26가지 종류

In [ ]:
#softmax_cross_entropy_with_logits
#label값을 one-hot 인코딩으로 넘겨줌
#sparse_softmax_cross_entropy_with_logits
#label값이 one-hot 인코딩이 안된 경우에 사용

In [ ]:
lr=0.01
n_hidden=128  #셀 출력
total_epoch=30 #에폭
n_step=3  #입력 3글자
n_input=n_class=dic_len #26, 입력 크기(n_input), 클래스 종류 개수(n_class)

In [ ]:
tf.reset_default_graph()

In [ ]:
#신경망 모델 구성
x = tf.placeholder(tf.float32, [None, n_step, n_input])
# None:전체 단어수, n_step:3글자 입력, n_input: 26개 종류
y = tf.placeholder(tf.int32, [None])
# y가 원핫 인코딩 되어 있다면 -> [None, n_class]
w = tf.Variable(tf.random_normal([n_hidden,n_class]))
b = tf.Variable(tf.random_normal([n_class]))
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell,x, dtype=tf.float32)

In [ ]:
# outputs shape : (10, 3, 128)
tf.transpose(outputs, [1,0,2])
# (10, 3, 128) => (3, 10, 128)
outputs = outputs[-1] #(10, 128)
model = tf.matmul(outputs, w) + b
# tf.matmul([10, 128],[128, 26]) => [10,26]     #[n_hidden, n_class] => [128, 26]

In [ ]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=y))
opt = tf.train.AdamOptimizer(lr).minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
#outputs을 실행하여 outputs의 shape 출력?
print(sess.run(outputs, feed_dict={x:input_batch}).shape)

In [ ]:
for epoch in range(total_epoch):
    _, cv = sess.run([opt,cost], feed_dict={x:input_batch, y:target_batch})
    print("에폭:", "%04d" %(epoch+1),"비용:","{:.5f}".format(cv))
print("모델 작성 완료")

In [ ]:
pred = tf.cast(tf.argmax(model,1), tf.int32)
predCheck = tf.equal(pred, y)
accuracy = tf.reduce_mean(tf.cast(predCheck, tf.float32))

input_batch, target_batch = make_batch(seq_data)
pv, av = sess.run([pred, accuracy], feed_dict={x:input_batch, y:target_batch})

In [ ]:
predict_words=[]
for i,v in enumerate(seq_data):
    last_char = char_arr[pv[i]]
    predict_words.append(v[:3]+last_char)
print("예측결과\n")
print("입력값:", [w[:3] for w in seq_data])
print("예측값:", predict_words)
pint("정확도:", av)

## seq2seq: 기계번역

In [ ]:
# 입력신경망(인코더)/출력신경망(디코더)
# 나는 학교에 간다 -> I go to school

#seq2seq: 챗봇, 번역, 이미지 캡셔닝
#영어 단어 -> 한국어 단어 번역기

In [ ]:
import numpy as np
import tensorflow as tf
"""
S: 디코딩 입력의 시작
E: 디코딩 출력의 끝
P: 현재 배치되는 데이터의 time step 크기보다 작은 경우, 빈 시퀀스를 채우는 심볼

배치 데이터의 최대 크기 4인 경우
word=> ['w','o','r','d']
to=> ['t','o', 'P', 'P']
"""
char_arr = [c for c in "SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑"]
num_dic = {n:i for i,n in enumerate(char_arr)}
dic_len = len(num_dic)

In [ ]:
seq_data = [['word','단어'],['wood','나무'],
            ['game','놀이'],['girl','소녀'],
            ['love','사랑'],['kiss','키스']]
#영어 -> 한글

In [ ]:
def make_batch(seq_date):
    input_batch=[]
    output_batch=[]
    target_batch=[]
    for seq in seq_data:
#         print(np.shape(seq_data)) #(6, 2)
        inputdata = [num_dic[n] for n in seq[0]] #word -> [25, 17, 20, 6]....
#         print(inputdata)
        outputdata = [num_dic[n] for n in ('S'+seq[1])] #한국어 단어에 대한 인덱스
        targetdata = [num_dic[n] for n in (seq[1]+'E')]
        
        input_batch.append(np.eye(dic_len)[inputdata])
        output_batch.append(np.eye(dic_len)[outputdata])
        target_batch.append(targetdata) #출력값만 원핫이 아님(sparse_ 사용)
    return input_batch, output_batch, target_batch

In [ ]:
input_batch, output_batch, target_batch = make_batch(seq_data)

In [ ]:
target_batch

In [ ]:
#모델 옵션 설정
lr=0.01
n_hidden=128
total_epoch=100
n_class=n_input=dic_len #41

In [ ]:
#신경망 구성 = [배치사이즈, 단계, 입력크기]
encInput = tf.placeholder(tf.float32, [None,None,n_input])  # 영어단어 (word)
decInput = tf.placeholder(tf.float32, [None,None,n_input]) # <S>단어
targets = tf.placeholder(tf.int64, [None,None])   # 단어<E>

In [ ]:
#인코더 셀을 구성
with tf.variable_scope("encode"):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, encInput, dtype=tf.float32)

In [ ]:
#디코더 셀 구성
with tf.variable_scope("decode"):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, decInput, initial_state=enc_states, dtype=tf.float32)

In [ ]:
model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
opt = tf.train.AdamOptimizer(lr).minimize(cost)

In [ ]:
#신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
input_batch, output_batch, target_batch = make_batch(seq_data)

In [ ]:
for epoch in range(total_epoch):
    _, cv = sess.run([opt,cost], feed_dict={encInput:input_batch, decInput:output_batch, 
                                            targets:target_batch})
    print("에폭:", "%04d" %(epoch+1),
         "비용:", "{:.5f}".format(cv))

In [ ]:
def translate(w):
    seq_data = [w, 'P'*len(w)]
    #seq_data = ['love', 'PPPP']
    input_batch, output_batch, target_batch = make_batch([seq_data])
    #input_batch=['w','o','r','d']
    #output_batch=['P','P','P','P']
    #target_batch=[2,2,2,2]
    
    #model 실행 결과: [배치사이즈, 스텝, 입력크기]
    #2번째 차원인 입력 차원을 argmax 적용 -> 확률이 가장 높은 글자 예측
    
    prediction = tf.argmax(model,2)
    #[[[0,0,0.9.....0.1]]]
    # => [[[2],[3]....]]  #글자 인덱스
    #['단','어','E','E']
    
    res = sess.run(prediction, feed_dict={encInput:input_batch, decInput:output_batch, 
                                          targets:target_batch})
#     res에는 숫자 인덱스
    decoded = [char_arr[i] for i in res[0]]
    end = decoded.index("E")
    translated = "".join(decoded[:end])
    return translated
    

In [ ]:
print("word?", translate("word"))
print("wodr?", translate("wodr"))
print("love?", translate("love"))
print("loev?", translate("loev"))
print("like?", translate("like"))